In [1]:
import numpy as np

In [2]:
#data = ["G503746","0XXXXX4","3XS6138","3XXXXX1","1X1973X","5XXXXX8","3375042"]
data = ["G95X823X101","4X1X7X2X0X8","6X237X569X2","0XXXXXXXXX0","6X858X41707","2X5XXX1XXXX","7X3X525XS68","4X7X9XXXXX6","5X4X6X33242","5X8X3X9XXXX","100X8096058"]
data = [list(line) for line in data]
data = np.array(data)
data

array([['G', '9', '5', 'X', '8', '2', '3', 'X', '1', '0', '1'],
       ['4', 'X', '1', 'X', '7', 'X', '2', 'X', '0', 'X', '8'],
       ['6', 'X', '2', '3', '7', 'X', '5', '6', '9', 'X', '2'],
       ['0', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', '0'],
       ['6', 'X', '8', '5', '8', 'X', '4', '1', '7', '0', '7'],
       ['2', 'X', '5', 'X', 'X', 'X', '1', 'X', 'X', 'X', 'X'],
       ['7', 'X', '3', 'X', '5', '2', '5', 'X', 'S', '6', '8'],
       ['4', 'X', '7', 'X', '9', 'X', 'X', 'X', 'X', 'X', '6'],
       ['5', 'X', '4', 'X', '6', 'X', '3', '3', '2', '4', '2'],
       ['5', 'X', '8', 'X', '3', 'X', '9', 'X', 'X', 'X', 'X'],
       ['1', '0', '0', 'X', '8', '0', '9', '6', '0', '5', '8']],
      dtype='<U1')

In [3]:
for i,row in enumerate(data):
    for j, spot in enumerate(row):
        if spot == "S":
            init_pos = (i,j)
            break
            
print(init_pos)

(6, 8)


In [4]:
data[init_pos[0],init_pos[1]]

'S'

In [5]:
def move_validation(move_dict):
    
    valid_move = []
    
    for key, value in move_dict.items():
        
        if(value[0] >=0 and value[1] >= 0 and value[0] <= 10 and value[1] <= 10):
 
            if(data[value] != "X"):
                
                if(sub_problems[-1]):
                    
                    if(sub_problems[-1][-1] == oposite_move[key]):
                        
                        continue
                    
                valid_move.append(key)
                
    return valid_move

In [6]:
oposite_move = {"up":"down", "down":"up", "right":"left", "left":"right"}
count=1

x,y = init_pos[0],init_pos[1]
move_dict = {"up":(x-1,y), "down":(x+1,y), "right":(x,y+1), "left":(x,y-1)}

sub_problems = [[]]
sub_problems_pos = [(x,y)]
sub_problems_sum = [0]

while (spot != 'G' and count<=250):
    
    x,y = sub_problems_pos[-1]
    move_dict = {"up":(x-1,y), "down":(x+1,y), "right":(x,y+1), "left":(x,y-1)}
    valid_move = move_validation(move_dict)
    
    # If there is only one valid move, make that move   
    if len(valid_move) == 1:
        
        print('foi')
        
        x,y = move_dict[valid_move[0]]
        spot = data[x,y]
        
        sub_problems[-1].append(valid_move[0])
        sub_problems_pos[-1] = (x,y)
        
        if spot != "G":
            sub_problems_sum[-1] += int(spot)
        
        #print(sub_problems)
    
    # If a bifurcation, store current path as a subproblem for each valid move
    elif len(valid_move) > 1: 

        fork_moves = sub_problems[-1].copy()
        fork_pos = sub_problems_pos[-1]
        fork_sum = sub_problems_sum[-1]
        
        sub_problems.pop()
        sub_problems_pos.pop()
        sub_problems_sum.pop()
        
        for move in valid_move:
            
            sub_problems.append(fork_moves + [move])
            sub_problems_pos.append((move_dict[move]))
            
            spot = data[move_dict[move]]
            sub_problems_sum.append(fork_sum + int(spot))
    
    elif len(valid_move) == 0:
        
        if len(sub_problems) == 1:
            print("No path found")
            print(sub_problems)
            print(sub_problems_pos)
            print(sub_problems_sum)
            print(count)
            break
            
        # If a dead end, backtrack to the last subproblem
        else:
            
            sub_problems.pop()
            sub_problems_pos.pop()
            x,y = sub_problems_pos[-1]
            continue

    count+=1

foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
aqui
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi


In [7]:
print(sub_problems)
print(sub_problems_pos)
print(sub_problems_sum)

[['right', 'right', 'down', 'down', 'left', 'left', 'left', 'left', 'down', 'down', 'right'], ['right', 'right', 'down', 'down', 'left', 'left', 'left', 'left', 'down', 'down', 'left', 'left', 'up', 'up', 'up', 'up', 'right', 'right', 'up', 'up', 'right', 'right', 'right', 'right', 'up', 'up', 'up', 'up', 'left', 'left', 'down', 'down', 'left', 'left', 'up', 'up', 'left', 'left', 'down', 'down', 'left', 'left', 'up', 'up', 'left', 'left']]
[(10, 7), (0, 0)]
[58, 191]
